In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### only flip ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 9s 44ms/step - loss: 2.2333 - accuracy: 0.3613 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 22ms/step - loss: 1.5868 - accuracy: 0.5120 - val_loss: 1.6702 - val_accuracy: 0.4928
Epoch 3/150
196/196 [==============================] - 4s 23ms/step - loss: 1.3589 - accuracy: 0.5909 - val_loss: 1.2306 - val_accuracy: 0.6252
Epoch 4/150
196/196 [==============================] - 4s 23ms/step - loss: 1.2364 - accuracy: 0.6345 - val_loss: 1.2227 - val_accuracy: 0.6453
Epoch 5/150
196/196 [==============================] - 4s 22ms/step - loss: 1.1098 - accuracy: 0.6738 - val_loss: 1.0624 - val_accuracy: 0.6913
Epoch 6/150
196/196 [==============================] - 4s 23ms/step - loss: 1.0125 - accuracy: 0.7028 - val_loss: 0.9262 - val_accuracy: 0.7367
Epoch 7/150
196/196 [==============================] - 4s 23ms/ste

196/196 [==============================] - 5s 23ms/step - loss: 0.5364 - accuracy: 0.9090 - val_loss: 0.6566 - val_accuracy: 0.8831
Epoch 58/150
196/196 [==============================] - 5s 23ms/step - loss: 0.5375 - accuracy: 0.9070 - val_loss: 0.6639 - val_accuracy: 0.8796
Epoch 59/150
196/196 [==============================] - 4s 23ms/step - loss: 0.5352 - accuracy: 0.9106 - val_loss: 0.6650 - val_accuracy: 0.8814
Epoch 60/150
196/196 [==============================] - 5s 23ms/step - loss: 0.5348 - accuracy: 0.9118 - val_loss: 0.6616 - val_accuracy: 0.8814
Epoch 61/150
196/196 [==============================] - 5s 23ms/step - loss: 0.5372 - accuracy: 0.9119 - val_loss: 0.6568 - val_accuracy: 0.8808
Epoch 62/150
196/196 [==============================] - 5s 24ms/step - loss: 0.5354 - accuracy: 0.9108 - val_loss: 0.6866 - val_accuracy: 0.8757
Epoch 63/150
196/196 [==============================] - 5s 24ms/step - loss: 0.5325 - accuracy: 0.9116 - val_loss: 0.6660 - val_accuracy: 0.879

In [6]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### remove data augmentation and  kernel_regularizer ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 6s 31ms/step - loss: 2.1173 - accuracy: 0.3620 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 23ms/step - loss: 1.4545 - accuracy: 0.4987 - val_loss: 1.6015 - val_accuracy: 0.4774
Epoch 3/150
196/196 [==============================] - 4s 23ms/step - loss: 1.2615 - accuracy: 0.5704 - val_loss: 1.2317 - val_accuracy: 0.5804
Epoch 4/150
196/196 [==============================] - 4s 23ms/step - loss: 1.1180 - accuracy: 0.6180 - val_loss: 1.4178 - val_accuracy: 0.6060
Epoch 5/150
196/196 [==============================] - 4s 23ms/step - loss: 0.9618 - accuracy: 0.6688 - val_loss: 0.8623 - val_accuracy: 0.7043
Epoch 6/150
196/196 [==============================] - 4s 22ms/step - loss: 0.8644 - accuracy: 0.7027 - val_loss: 0.7746 - val_accuracy: 0.7303
Epoch 7/150
196/196 [==============================] - 4s 23ms/ste

196/196 [==============================] - 4s 23ms/step - loss: 0.1628 - accuracy: 0.9423 - val_loss: 0.4945 - val_accuracy: 0.8608
Epoch 58/150
196/196 [==============================] - 5s 23ms/step - loss: 0.1707 - accuracy: 0.9405 - val_loss: 0.4893 - val_accuracy: 0.8617
Epoch 59/150
196/196 [==============================] - 4s 23ms/step - loss: 0.1561 - accuracy: 0.9443 - val_loss: 0.5143 - val_accuracy: 0.8643
Epoch 60/150
196/196 [==============================] - 4s 23ms/step - loss: 0.1553 - accuracy: 0.9446 - val_loss: 0.4934 - val_accuracy: 0.8657
Epoch 61/150
196/196 [==============================] - 4s 23ms/step - loss: 0.1475 - accuracy: 0.9485 - val_loss: 0.4991 - val_accuracy: 0.8661
Epoch 62/150
196/196 [==============================] - 4s 23ms/step - loss: 0.1465 - accuracy: 0.9477 - val_loss: 0.5275 - val_accuracy: 0.8592
Epoch 63/150
196/196 [==============================] - 4s 23ms/step - loss: 0.1535 - accuracy: 0.9463 - val_loss: 0.5210 - val_accuracy: 0.861

In [9]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### remove data augmentation and  kernel_regularizer and use SpatialDropout2D ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(SpatialDropout2D(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(SpatialDropout2D(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(SpatialDropout2D(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(SpatialDropout2D(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 6s 30ms/step - loss: 2.1415 - accuracy: 0.3238 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 22ms/step - loss: 1.6122 - accuracy: 0.4467 - val_loss: 1.3582 - val_accuracy: 0.5136
Epoch 3/150
196/196 [==============================] - 4s 22ms/step - loss: 1.4101 - accuracy: 0.5167 - val_loss: 1.1795 - val_accuracy: 0.5814
Epoch 4/150
196/196 [==============================] - 4s 22ms/step - loss: 1.2545 - accuracy: 0.5714 - val_loss: 1.0165 - val_accuracy: 0.6418
Epoch 5/150
196/196 [==============================] - 4s 22ms/step - loss: 1.1359 - accuracy: 0.6159 - val_loss: 0.9637 - val_accuracy: 0.6586
Epoch 6/150
196/196 [==============================] - 4s 22ms/step - loss: 1.0234 - accuracy: 0.6491 - val_loss: 0.8371 - val_accuracy: 0.7041
Epoch 7/150
196/196 [==============================] - 4s 22ms/ste

196/196 [==============================] - 5s 23ms/step - loss: 0.2203 - accuracy: 0.9211 - val_loss: 0.5506 - val_accuracy: 0.8493
Epoch 58/150
196/196 [==============================] - 4s 23ms/step - loss: 0.2142 - accuracy: 0.9236 - val_loss: 0.5303 - val_accuracy: 0.8506
Epoch 59/150
196/196 [==============================] - 5s 23ms/step - loss: 0.2127 - accuracy: 0.9250 - val_loss: 0.5414 - val_accuracy: 0.8475
Epoch 60/150
196/196 [==============================] - 4s 23ms/step - loss: 0.2059 - accuracy: 0.9258 - val_loss: 0.5407 - val_accuracy: 0.8512
Epoch 61/150
196/196 [==============================] - 5s 23ms/step - loss: 0.2011 - accuracy: 0.9284 - val_loss: 0.5476 - val_accuracy: 0.8522
Epoch 62/150
196/196 [==============================] - 4s 23ms/step - loss: 0.2021 - accuracy: 0.9289 - val_loss: 0.5491 - val_accuracy: 0.8493
Epoch 63/150
196/196 [==============================] - 4s 23ms/step - loss: 0.1966 - accuracy: 0.9296 - val_loss: 0.5763 - val_accuracy: 0.847

In [12]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### remove scheduler ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

opt_adam  = keras.optimizers.Adam(learning_rate=0.001)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 6s 33ms/step - loss: 2.3610 - accuracy: 0.3314 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 5s 23ms/step - loss: 1.7269 - accuracy: 0.4648 - val_loss: 1.8528 - val_accuracy: 0.4368
Epoch 3/150
196/196 [==============================] - 5s 23ms/step - loss: 1.5415 - accuracy: 0.5343 - val_loss: 1.2822 - val_accuracy: 0.6068
Epoch 4/150
196/196 [==============================] - 5s 23ms/step - loss: 1.3677 - accuracy: 0.5911 - val_loss: 1.2307 - val_accuracy: 0.6270
Epoch 5/150
196/196 [==============================] - 5s 23ms/step - loss: 1.2235 - accuracy: 0.6370 - val_loss: 1.0490 - val_accuracy: 0.6962
Epoch 6/150
196/196 [==============================] - 5s 23ms/step - loss: 1.1177 - accuracy: 0.6752 - val_loss: 0.9917 - val_accuracy: 0.7114
Epoch 7/150
196/196 [==============================] - 5s 23ms/ste

196/196 [==============================] - 5s 23ms/step - loss: 0.6155 - accuracy: 0.8772 - val_loss: 0.6113 - val_accuracy: 0.8852
Epoch 58/150
196/196 [==============================] - 5s 23ms/step - loss: 0.6152 - accuracy: 0.8769 - val_loss: 0.6141 - val_accuracy: 0.8846
Epoch 59/150
196/196 [==============================] - 5s 23ms/step - loss: 0.6068 - accuracy: 0.8816 - val_loss: 0.6005 - val_accuracy: 0.8855
Epoch 60/150
196/196 [==============================] - 5s 23ms/step - loss: 0.6078 - accuracy: 0.8794 - val_loss: 0.6114 - val_accuracy: 0.8846
Epoch 61/150
196/196 [==============================] - 5s 23ms/step - loss: 0.6034 - accuracy: 0.8823 - val_loss: 0.6500 - val_accuracy: 0.8755
Epoch 62/150
196/196 [==============================] - 5s 23ms/step - loss: 0.6072 - accuracy: 0.8818 - val_loss: 0.6113 - val_accuracy: 0.8839
Epoch 63/150
196/196 [==============================] - 5s 23ms/step - loss: 0.6107 - accuracy: 0.8814 - val_loss: 0.6168 - val_accuracy: 0.882

In [17]:
# add skip connection
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - add skip connection ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

# model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

# block 1
previous = x

x = Conv2D(32, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 2
previous = x

x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 3
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# block 4
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# dense block 

x = Flatten()(x)
x = Dense(128)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 7s 37ms/step - loss: 4.4943 - accuracy: 0.2801 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 5s 25ms/step - loss: 2.3163 - accuracy: 0.3850 - val_loss: 2.3226 - val_accuracy: 0.3125
Epoch 3/150
196/196 [==============================] - 5s 25ms/step - loss: 2.0566 - accuracy: 0.4348 - val_loss: 2.0705 - val_accuracy: 0.3855
Epoch 4/150
196/196 [==============================] - 5s 25ms/step - loss: 1.8473 - accuracy: 0.4820 - val_loss: 1.6044 - val_accuracy: 0.5196
Epoch 5/150
196/196 [==============================] - 5s 25ms/step - loss: 1.6977 - accuracy: 0.5183 - val_loss: 1.4908 - val_accuracy: 0.5559
Epoch 6/150
196/196 [==============================] - 5s 25ms/step - loss: 1.5332 - accuracy: 0.5569 - val_loss: 1.3937 - val_accuracy: 0.5989
Epoch 7/150
196/196 [==============================] - 5s 25ms/ste

196/196 [==============================] - 5s 26ms/step - loss: 0.6380 - accuracy: 0.8777 - val_loss: 0.6488 - val_accuracy: 0.8796
Epoch 58/150
196/196 [==============================] - 5s 26ms/step - loss: 0.6421 - accuracy: 0.8763 - val_loss: 0.6642 - val_accuracy: 0.8770
Epoch 59/150
196/196 [==============================] - 5s 26ms/step - loss: 0.6407 - accuracy: 0.8774 - val_loss: 0.6846 - val_accuracy: 0.8696
Epoch 60/150
196/196 [==============================] - 5s 26ms/step - loss: 0.6380 - accuracy: 0.8777 - val_loss: 0.6545 - val_accuracy: 0.8830
Epoch 61/150
196/196 [==============================] - 5s 26ms/step - loss: 0.6384 - accuracy: 0.8784 - val_loss: 0.6848 - val_accuracy: 0.8734
Epoch 62/150
196/196 [==============================] - 5s 26ms/step - loss: 0.6383 - accuracy: 0.8784 - val_loss: 0.6615 - val_accuracy: 0.8820
Epoch 63/150
196/196 [==============================] - 5s 26ms/step - loss: 0.6385 - accuracy: 0.8774 - val_loss: 0.6727 - val_accuracy: 0.879

In [20]:
# add skip connection
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - add skip connection and increase filters ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

# model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

# block 1
previous = x

x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 2
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 3
previous = x

x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# block 4
previous = x

x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# dense block 

x = Flatten()(x)
x = Dense(128)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 16s 82ms/step - loss: 6.5520 - accuracy: 0.2791 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 11s 56ms/step - loss: 3.7520 - accuracy: 0.3721 - val_loss: 4.2107 - val_accuracy: 0.2850
Epoch 3/150
196/196 [==============================] - 11s 57ms/step - loss: 3.2166 - accuracy: 0.4329 - val_loss: 3.0564 - val_accuracy: 0.4158
Epoch 4/150
196/196 [==============================] - 11s 57ms/step - loss: 2.9238 - accuracy: 0.4807 - val_loss: 2.7466 - val_accuracy: 0.4234
Epoch 5/150
196/196 [==============================] - 11s 57ms/step - loss: 2.5212 - accuracy: 0.5269 - val_loss: 2.3957 - val_accuracy: 0.5306
Epoch 6/150
196/196 [==============================] - 11s 57ms/step - loss: 2.2671 - accuracy: 0.5649 - val_loss: 3.1948 - val_accuracy: 0.5554
Epoch 7/150
196/196 [==============================] - 11s 5

Epoch 57/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8251 - accuracy: 0.8987 - val_loss: 0.9271 - val_accuracy: 0.8775
Epoch 58/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8302 - accuracy: 0.8969 - val_loss: 0.9926 - val_accuracy: 0.8642
Epoch 59/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8277 - accuracy: 0.8976 - val_loss: 0.9768 - val_accuracy: 0.8695
Epoch 60/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8251 - accuracy: 0.8986 - val_loss: 0.8882 - val_accuracy: 0.8899
Epoch 61/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8135 - accuracy: 0.9011 - val_loss: 0.8748 - val_accuracy: 0.8903
Epoch 62/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8126 - accuracy: 0.9015 - val_loss: 0.8990 - val_accuracy: 0.8892
Epoch 63/150
196/196 [==============================] - 11s 55ms/step - loss: 0.8105 - accuracy: 0.9011 - val_loss: 0.8894 -

196/196 [==============================] - 11s 55ms/step - loss: 0.4894 - accuracy: 0.9601 - val_loss: 0.6666 - val_accuracy: 0.9219
Epoch 114/150
196/196 [==============================] - 11s 54ms/step - loss: 0.4779 - accuracy: 0.9629 - val_loss: 0.6629 - val_accuracy: 0.9239
Epoch 115/150
196/196 [==============================] - 11s 54ms/step - loss: 0.4737 - accuracy: 0.9625 - val_loss: 0.6652 - val_accuracy: 0.9227
Epoch 116/150
196/196 [==============================] - 11s 54ms/step - loss: 0.4711 - accuracy: 0.9627 - val_loss: 0.6580 - val_accuracy: 0.9234
Epoch 117/150
196/196 [==============================] - 11s 54ms/step - loss: 0.4642 - accuracy: 0.9630 - val_loss: 0.6602 - val_accuracy: 0.9231
Epoch 118/150
196/196 [==============================] - 11s 54ms/step - loss: 0.4634 - accuracy: 0.9637 - val_loss: 0.6544 - val_accuracy: 0.9226
Epoch 119/150
196/196 [==============================] - 11s 54ms/step - loss: 0.4560 - accuracy: 0.9645 - val_loss: 0.6482 - val_ac

In [23]:
# add skip connection
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - add skip connection and increase filters ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

# model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

# block 1
previous = x

x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 2
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 3
previous = x

x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# block 4
previous = x

x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = BatchNormalization()(x)

x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# dense block 

x = Flatten()(x)
x = Dense(128)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 19s 95ms/step - loss: 6.5713 - accuracy: 0.2699 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 15s 79ms/step - loss: 4.2456 - accuracy: 0.3633 - val_loss: 3.2400 - val_accuracy: 0.3343
Epoch 3/150
196/196 [==============================] - 15s 79ms/step - loss: 4.1170 - accuracy: 0.3722 - val_loss: 2.9031 - val_accuracy: 0.3811
Epoch 4/150
196/196 [==============================] - 15s 79ms/step - loss: 3.2170 - accuracy: 0.4476 - val_loss: 2.8963 - val_accuracy: 0.4259
Epoch 5/150
196/196 [==============================] - 16s 81ms/step - loss: 2.8477 - accuracy: 0.4944 - val_loss: 2.7399 - val_accuracy: 0.4124
Epoch 6/150
196/196 [==============================] - 16s 80ms/step - loss: 2.5309 - accuracy: 0.5349 - val_loss: 2.5163 - val_accuracy: 0.4572
Epoch 7/150
196/196 [==============================] - 16s 7

Epoch 57/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7904 - accuracy: 0.9087 - val_loss: 0.9110 - val_accuracy: 0.8837
Epoch 58/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7844 - accuracy: 0.9119 - val_loss: 0.8724 - val_accuracy: 0.8919
Epoch 59/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7805 - accuracy: 0.9120 - val_loss: 0.9159 - val_accuracy: 0.8800
Epoch 60/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7661 - accuracy: 0.9177 - val_loss: 0.8871 - val_accuracy: 0.8864
Epoch 61/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7723 - accuracy: 0.9157 - val_loss: 0.9040 - val_accuracy: 0.8816
Epoch 62/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7610 - accuracy: 0.9165 - val_loss: 0.9054 - val_accuracy: 0.8847
Epoch 63/150
196/196 [==============================] - 16s 83ms/step - loss: 0.7635 - accuracy: 0.9182 - val_loss: 0.8407 -

196/196 [==============================] - 16s 82ms/step - loss: 0.4300 - accuracy: 0.9767 - val_loss: 0.6708 - val_accuracy: 0.9233
Epoch 114/150
196/196 [==============================] - 16s 82ms/step - loss: 0.4271 - accuracy: 0.9757 - val_loss: 0.6701 - val_accuracy: 0.9235
Epoch 115/150
196/196 [==============================] - 16s 82ms/step - loss: 0.4188 - accuracy: 0.9770 - val_loss: 0.6645 - val_accuracy: 0.9258
Epoch 116/150
196/196 [==============================] - 16s 82ms/step - loss: 0.4139 - accuracy: 0.9775 - val_loss: 0.6686 - val_accuracy: 0.9243
Epoch 117/150
196/196 [==============================] - 16s 82ms/step - loss: 0.4095 - accuracy: 0.9770 - val_loss: 0.6729 - val_accuracy: 0.9243
Epoch 118/150
196/196 [==============================] - 16s 83ms/step - loss: 0.4052 - accuracy: 0.9778 - val_loss: 0.6685 - val_accuracy: 0.9236
Epoch 119/150
196/196 [==============================] - 16s 82ms/step - loss: 0.4013 - accuracy: 0.9785 - val_loss: 0.6681 - val_ac

In [24]:
# add skip connection
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - add skip connection and increase filters ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

# model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

# block 1
previous = x

x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 2
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 3
previous = x

x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# block 4
previous = x

x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# dense block 

x = Flatten()(x)
x = Dense(128)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

#training
batch_size = 256
epochs = 250
steps = x_train.shape[0] // batch_size

boundaries = [steps*80, steps*160]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/250
196/196 [==============================] - 31s 157ms/step - loss: 4.1578 - accuracy: 0.3210 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/250
196/196 [==============================] - 24s 122ms/step - loss: 2.7453 - accuracy: 0.4806 - val_loss: 2.6180 - val_accuracy: 0.5345
Epoch 3/250
196/196 [==============================] - 24s 123ms/step - loss: 2.4596 - accuracy: 0.5707 - val_loss: 2.3524 - val_accuracy: 0.6057
Epoch 4/250
196/196 [==============================] - 24s 123ms/step - loss: 2.2483 - accuracy: 0.6306 - val_loss: 2.1200 - val_accuracy: 0.6803
Epoch 5/250
196/196 [==============================] - 24s 123ms/step - loss: 2.0865 - accuracy: 0.6758 - val_loss: 1.9367 - val_accuracy: 0.7273
Epoch 6/250
196/196 [==============================] - 24s 123ms/step - loss: 1.9403 - accuracy: 0.7137 - val_loss: 1.7931 - val_accuracy: 0.7585
Epoch 7/250
196/196 [==============================] -

Epoch 112/250
196/196 [==============================] - 23s 117ms/step - loss: 0.6097 - accuracy: 0.9607 - val_loss: 0.7928 - val_accuracy: 0.9190
Epoch 113/250
196/196 [==============================] - 23s 117ms/step - loss: 0.6116 - accuracy: 0.9600 - val_loss: 0.7791 - val_accuracy: 0.9183
Epoch 114/250
196/196 [==============================] - 23s 117ms/step - loss: 0.6053 - accuracy: 0.9611 - val_loss: 0.7962 - val_accuracy: 0.9171
Epoch 115/250
196/196 [==============================] - 23s 117ms/step - loss: 0.6040 - accuracy: 0.9608 - val_loss: 0.7829 - val_accuracy: 0.9167
Epoch 116/250
196/196 [==============================] - 23s 117ms/step - loss: 0.6000 - accuracy: 0.9621 - val_loss: 0.7869 - val_accuracy: 0.9185
Epoch 117/250
196/196 [==============================] - 23s 117ms/step - loss: 0.5986 - accuracy: 0.9623 - val_loss: 0.7844 - val_accuracy: 0.9176
Epoch 118/250
196/196 [==============================] - 23s 117ms/step - loss: 0.6031 - accuracy: 0.9609 - val_

196/196 [==============================] - 23s 117ms/step - loss: 0.3148 - accuracy: 0.9903 - val_loss: 0.5894 - val_accuracy: 0.9345
Epoch 223/250
196/196 [==============================] - 23s 117ms/step - loss: 0.3131 - accuracy: 0.9909 - val_loss: 0.6002 - val_accuracy: 0.9327
Epoch 224/250
196/196 [==============================] - 23s 117ms/step - loss: 0.3134 - accuracy: 0.9902 - val_loss: 0.5987 - val_accuracy: 0.9330
Epoch 225/250
196/196 [==============================] - 23s 117ms/step - loss: 0.3132 - accuracy: 0.9897 - val_loss: 0.5976 - val_accuracy: 0.9340
Epoch 226/250
196/196 [==============================] - 23s 117ms/step - loss: 0.3117 - accuracy: 0.9900 - val_loss: 0.5908 - val_accuracy: 0.9323
Epoch 227/250
196/196 [==============================] - 23s 117ms/step - loss: 0.3117 - accuracy: 0.9901 - val_loss: 0.5832 - val_accuracy: 0.9351
Epoch 228/250
196/196 [==============================] - 23s 117ms/step - loss: 0.3089 - accuracy: 0.9904 - val_loss: 0.5982 -

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code-add instance norm and increase filters###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 23s 120ms/step - loss: 3.5215 - accuracy: 0.3384 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 16s 82ms/step - loss: 2.2801 - accuracy: 0.4887 - val_loss: 2.0413 - val_accuracy: 0.5549
Epoch 3/150
196/196 [==============================] - 16s 82ms/step - loss: 1.9619 - accuracy: 0.5828 - val_loss: 1.9595 - val_accuracy: 0.6041
Epoch 4/150
196/196 [==============================] - 16s 82ms/step - loss: 1.7581 - accuracy: 0.6471 - val_loss: 1.6226 - val_accuracy: 0.6984
Epoch 5/150
196/196 [==============================] - 16s 82ms/step - loss: 1.6136 - accuracy: 0.6945 - val_loss: 1.4830 - val_accuracy: 0.7411
Epoch 6/150
196/196 [==============================] - 16s 82ms/step - loss: 1.4916 - accuracy: 0.7302 - val_loss: 1.3730 - val_accuracy: 0.7725
Epoch 7/150
196/196 [==============================] - 16s 

Epoch 57/150
196/196 [==============================] - 15s 78ms/step - loss: 1.0232 - accuracy: 0.9119 - val_loss: 1.0887 - val_accuracy: 0.8983
Epoch 58/150
196/196 [==============================] - 15s 78ms/step - loss: 1.0240 - accuracy: 0.9101 - val_loss: 1.0840 - val_accuracy: 0.8985
Epoch 59/150
196/196 [==============================] - 15s 78ms/step - loss: 1.0106 - accuracy: 0.9137 - val_loss: 1.0945 - val_accuracy: 0.8949
Epoch 60/150
196/196 [==============================] - 15s 78ms/step - loss: 1.0160 - accuracy: 0.9108 - val_loss: 1.0645 - val_accuracy: 0.9008
Epoch 61/150
196/196 [==============================] - 15s 78ms/step - loss: 1.0036 - accuracy: 0.9130 - val_loss: 1.0669 - val_accuracy: 0.8997
Epoch 62/150
196/196 [==============================] - 15s 78ms/step - loss: 0.9982 - accuracy: 0.9139 - val_loss: 1.0656 - val_accuracy: 0.8995
Epoch 63/150
196/196 [==============================] - 15s 77ms/step - loss: 1.0017 - accuracy: 0.9119 - val_loss: 1.1135 -

196/196 [==============================] - 15s 77ms/step - loss: 0.5775 - accuracy: 0.9691 - val_loss: 0.7400 - val_accuracy: 0.9274
Epoch 114/150
196/196 [==============================] - 15s 77ms/step - loss: 0.5687 - accuracy: 0.9698 - val_loss: 0.7350 - val_accuracy: 0.9278
Epoch 115/150
196/196 [==============================] - 15s 77ms/step - loss: 0.5600 - accuracy: 0.9705 - val_loss: 0.7293 - val_accuracy: 0.9281
Epoch 116/150
196/196 [==============================] - 15s 77ms/step - loss: 0.5510 - accuracy: 0.9714 - val_loss: 0.7234 - val_accuracy: 0.9316
Epoch 117/150
196/196 [==============================] - 15s 77ms/step - loss: 0.5441 - accuracy: 0.9714 - val_loss: 0.7163 - val_accuracy: 0.9323
Epoch 118/150
196/196 [==============================] - 15s 77ms/step - loss: 0.5356 - accuracy: 0.9736 - val_loss: 0.7091 - val_accuracy: 0.9322
Epoch 119/150
196/196 [==============================] - 15s 77ms/step - loss: 0.5307 - accuracy: 0.9725 - val_loss: 0.7058 - val_ac

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code-add instance norm and increase filters###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 31s 159ms/step - loss: 4.0937 - accuracy: 0.3266 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 23s 117ms/step - loss: 2.7242 - accuracy: 0.4888 - val_loss: 2.4925 - val_accuracy: 0.5569
Epoch 3/150
196/196 [==============================] - 23s 117ms/step - loss: 2.4390 - accuracy: 0.5709 - val_loss: 2.3855 - val_accuracy: 0.5988
Epoch 4/150
196/196 [==============================] - 23s 118ms/step - loss: 2.2289 - accuracy: 0.6313 - val_loss: 2.1023 - val_accuracy: 0.6802
Epoch 5/150
196/196 [==============================] - 23s 117ms/step - loss: 2.0484 - accuracy: 0.6809 - val_loss: 1.9407 - val_accuracy: 0.7168
Epoch 6/150
196/196 [==============================] - 23s 117ms/step - loss: 1.9071 - accuracy: 0.7162 - val_loss: 1.7980 - val_accuracy: 0.7528
Epoch 7/150
196/196 [==============================] -

In [2]:
import numpy as np
accs = np.array(history.history['accuracy'])*100
for i in range(150):
    print(accs[i])


33.84
48.868
58.28
64.706
69.454
73.024
75.766
77.37
78.548004
80.228004
81.423996
82.228
82.96
83.488
84.0
84.612
84.776
85.346
85.608
85.712
86.156
86.578
86.703995
87.158
87.27
87.466
87.926
88.042
87.956
88.173996
88.56
88.71
88.83
88.73
89.212
89.130005
89.282
89.340004
89.584
89.462
89.638
90.038
90.008
90.166
90.084
90.385994
90.628
90.486
90.582
90.521996
90.602
90.75
90.942
90.948
91.148
90.836
91.189995
91.008
91.369995
91.084
91.299995
91.392
91.194
91.228004
91.6
91.702
91.464005
91.788
91.74
92.136
93.654
94.124
94.270004
94.22
94.356
94.344
94.366005
94.352
94.464
94.492
94.774
94.404
94.706
94.68
94.602
94.631996
94.868
94.771996
94.658
94.794
94.942
94.824
94.86
94.76601
94.812
94.948
94.866005
94.951996
94.950005
94.786
95.034
95.178
95.038
95.032
95.084
95.058
95.088
95.216
95.14
95.422
96.41
96.652
96.914
96.979996
97.052
97.142
97.136
97.36
97.25
97.314
97.422
97.366
97.448
97.579994
97.516
97.425995
97.464005
97.656
97.71
97.668
97.626
97.566
97.672
97.82
97.826
97

In [3]:
val_as = np.array(history.history['val_accuracy'])*100
for i in range(150):
    print(val_as[i])

43.51
55.489998
60.41
69.840004
74.11
77.25
78.68
80.549995
77.85
81.27
83.350006
82.98
83.22
82.68
84.45
84.95
85.27
85.479996
85.07
85.5
87.12
86.0
87.07
86.67
85.17
87.11
87.4
86.85
87.46
87.75
87.840004
86.409996
87.23
87.08
87.46
87.85
88.810005
88.950005
88.54
88.14
88.31
88.590004
86.71
89.08
87.8
88.36
89.12
89.33
89.15
89.060005
89.1
89.32
89.270004
89.58
90.100006
89.51
89.83
89.850006
89.490005
90.08
89.97
89.950005
88.69
89.62
89.35
90.270004
89.560005
90.090004
90.47
91.1
91.5
91.7
91.689995
91.43
91.57
91.11
91.9
91.57
91.729996
91.27
91.28999
91.79
91.659996
91.54
91.72
92.15
91.649994
91.86
91.09
91.82
91.549995
91.61
91.119995
91.229996
91.32
91.229996
91.28
91.799995
91.84
92.049995
91.82
91.61
91.28
92.159996
91.36
91.27
91.909996
91.729996
91.85
92.409996
92.77
92.89
92.74
92.78
92.81
93.159996
93.229996
93.22
93.07
93.09
93.28
93.19
93.14
93.3
93.13
93.25
93.2
93.229996
92.97
93.01
93.09
93.04
93.03
92.93
92.89
92.89
93.1
93.15
93.19
93.14
93.11
93.14
93.11
92.92
9